## Make Profile summaries using Jinja2 and Python Model

*USE Python 3.7 to maintain order of Json files*

- Fetch SD file from IG
- Spec_internals from IG Package.tgz
- Create and use import Title map from ?Package?
- Transform to Python model
- use Jinja2 template to create a summary markdown file
- save markdown file



*Note need a successful build to generate since based on ig output local file
alternatively use package.files to generate*

### import python modules including R4 fhirclient models

In [36]:
from fhirclient.r4models import structuredefinition as SD
from fhirclient.r4models import narrative as N
from fhirclient.r4models import valueset as VS
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
import fhirclient.r4models.fhirreference as FR
from json import dumps, loads, load
from pprint import pprint
from jinja2 import Environment, FileSystemLoader, select_autoescape
from commonmark import commonmark
from IPython.display import display, HTML, Markdown
import title_map as tm  # title_map generated from 
import os
from stringcase import snakecase, titlecase
from pathlib import Path

### Some Globals

In [37]:
summ_elements =[
       'label',
       'short',
       'min',
       'max',
       'type',
       'binding',
       'sliceName',
        ]

choice_types = {'valueQuantity': 'value[x]',
                'valueCodeableConcept': 'value[x]',
                'valueString': 'value[x]',
                'valueInteger': 'value[x]',
                'valueDecimal': 'value[x]',
                'valueDateTime': 'value[x]',
                'valueRange': 'value[x]',
                'valuePeriod': 'value[x]',
                'dueDuration': 'due[x]',
                'dueDate': 'due[x]',
                'effectivedateTime': 'effective[x]',
                'effectivePeriod': 'effective[x]',
               } 

### Get file and return as dict

In [38]:
#ig_output_path =  "//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/output"
#ig_output_path =  "/Users/ehaas/Documents/FHIR/US-Core-R4/output"
#ig_source_path = "//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/source/"
#ig_source_path = "/Users/ehaas/Documents/FHIR/US-Core-R4/source/"
#ig_source_path = ''
ig_output_path =  "//ERICS-AIR-2/ehaas/Documents/FHIR/DaVinci-Notifications/output"
ig_source_path = "//ERICS-AIR-2/ehaas/Documents/FHIR/DaVinci-Notifications/source/"

def open_file(in_path, f_name): # get files
    with open(f'{in_path}/{f_name}', encoding="utf8") as f:
        r = f.read()
        return(loads(r))
   

### Get spec_internal from package.tgz a json file which includes canonical to local relative page links

In [39]:
import tarfile

def get_si(path):
    with tarfile.open(name=os.path.join(path,'package.tgz'), mode='r') as tf:
        #pprint(tf.getnames())
        f = tf.extractfile('other/spec.internals')
        r = f.read()
        return(loads(r))
    
       
si = get_si(ig_output_path) #spec internals
path_map = si['paths']

#  map  VS url to titles
title_map = {}

vs_files = [x for x in os.listdir(ig_output_path) if x.startswith("ValueSet") and x.endswith('.json')]
for i in vs_files:
    path = Path.cwd() / ig_output_path / i
    vs_json = path.read_text(encoding='utf8')
    vs = VS.ValueSet(loads(vs_json))
    title_map[vs.url]= vs.title if vs.title else 'None' # save in pages folder
     
title_map.update(tm.title_map) #add core to ig title maps

#path_map
title_map

{'http://hl7.org/fhir/us/davinci-notifications/ValueSet/notification-event': 'Da Vinci Notification Event ValueSet',
 'http://terminology.hl7.org/ValueSet/v3-RoleClassAssociative': 'V3 Value SetRoleClassAssociative',
 'http://terminology.hl7.org/ValueSet/v2-0474': 'v2 Practitioner Organization Unit Type',
 'http://hl7.org/fhir/ValueSet/device-status-reason': 'FHIRDeviceStatusReason',
 'http://hl7.org/fhir/ValueSet/definition-resource-types': 'DefinitionResourceType',
 'http://hl7.org/fhir/ValueSet/bodystructure-relative-location': 'Bodystructure Location Qualifier',
 'http://terminology.hl7.org/ValueSet/v2-0927': 'v2 Arm Stick',
 'http://hl7.org/fhir/ValueSet/encounter-status': 'EncounterStatus',
 'http://terminology.hl7.org/ValueSet/v3-ParticipationTargetDirect': 'V3 Value SetParticipationTargetDirect',
 'http://terminology.hl7.org/ValueSet/v3-ProvenanceEventCurrentState-AS': 'V3 Value SetProvenanceEventCurrentState-AS',
 'http://terminology.hl7.org/ValueSet/v2-0161': 'v2 ALLOW SUBSTI

### Using Jinja2 Template create md file for summary view 
*using Markdown instead of html for easier hand editing, the line spacing is resolved after template is rendered.*

uses these elements:
 -   'label',
 -   'short',
 -  'min',
 -  'max',
 -   'type',
 -   'binding',
 
 plus:
 an invariant list
 a hash table of value set urls to valueset titles.


In [40]:
def get_summary(profile_id,diff,constraints):

    in_path = ''
    template_path = 'summary-template.j2'
    core_path = 'http://hl7.org/fhir/R4/'

    bindings = dict(
        required = f'{core_path}terminologies.html#required',
        extensible = f'{core_path}terminologies.html#extensible',
        preferred =f'{core_path}terminologies.html#preferred',
        example = f'{core_path}terminologies.html#example',
    )
    


    def markdown(text, *args, **kwargs):
        return commonmark(text, *args, **kwargs)

    env = Environment(
        loader=FileSystemLoader(searchpath = in_path),
        autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
        )

    env.filters['markdown'] = markdown

    template = env.get_template(template_path)
    d = template.render(elements = diff, title_map=title_map, bindings=bindings, constraints=constraints, path_map = path_map)

    print(f'============== file_name = {profile_id}-summary.md ================')
    display(Markdown(d))
    return d

### For Creation of Summaries for a Whole Folder of StructureDefinitions (Profiles)

if want to create a summary for single file then uncomment out the next cell by changing to 'raw' or 'Markdown'

In [41]:
#in_path = '/Users/ehaas/.fhir/packages/hl7.fhir.us.core.r4#dev/package/' # tried to use this but package file keeps disappearing
files = [x for x in os.listdir(ig_output_path) if x.startswith("StructureDefinition") and x.endswith('json')]
files

['StructureDefinition-notifications-messageheader.json',
 'StructureDefinition-notifications-graphdefinition.json',
 'StructureDefinition-notifications-messageheader-src.json',
 'StructureDefinition-discharge-notification-messageheader.json',
 'StructureDefinition-admit-notification-messageheader.json',
 'StructureDefinition-extension-mustSupport.json',
 'StructureDefinition-admit-discharge-notification-coverage.json',
 'StructureDefinition-notifications-bundle.json',
 'StructureDefinition-admit-discharge-notification-encounter.json',
 'StructureDefinition-notifications-messagedefinition.json',
 'StructureDefinition-admit-discharge-notification-condition.json']

### for summary of single profile

if want to create a summary for a whole folder then comment out the following lines by changing to 'raw' or 'Markdown'

f_name = 'provenance'
files = [f'StructureDefinition-us-core-{f_name}.json']
files

### Loop through profiles and update missing stuff in Differential with Snapshot then generate Markdown summary file and save

In [42]:
for i in files:
    
    constraints = {}
    sd_dict = open_file(ig_output_path,i)
    sd = SD.StructureDefinition(sd_dict)
    profile_id = sd.id
    print(f'========{profile_id}=======')
    for i in sd.differential.element:
        path = i.path
        c = i.constraint
        print(f'====Path = {path} =====')
        # GET Invariant Dict  path: human readable invariant list.
        try:
            k = next((k for k in sd.snapshot.element if k.path == path))
            constraint = [j.human for j in k.constraint if 'dom-' not in j.key 
                          and 'ele-' not in j.key  and 'ext-' not in j.key]
            
        except TypeError:
            constraint = []
        except StopIteration:
            constraint = []
        if constraint:
            constraints[path]=constraint 

        for k in summ_elements:
            #print(f'differential = {path}.{k} = {getattr(i,k)}')
            if getattr(i,k) == None:
                try:
                    snap_element = (s for s in sd.snapshot.element if s.path == path)           
                    new_val = getattr(next(snap_element),k)
                    #print(f'snapshot = {path}.{k} = {new_val}')
                    setattr(i,k,new_val)
                except StopIteration: # assume is an choice data type
                    print(f'no snapshot element for {path}.{k} = {getattr(i,k)} assume is an choice data type')
                    new_plist = []
                    for p in path.split('.'):
                        try:
                            new_plist.append(choice_types[p])
                        except KeyError:
                            new_plist.append(p)
                    new_path = '.'.join(new_plist)
                    print(path,new_path)
                    snap_element = (s for s in sd.snapshot.element if s.path == new_path)
                    #print(list(snap_element))
                    try:
                        new_val = getattr(next(snap_element),k)
                        #print(f'snapshot = {path}.{k} = {new_val}')
                        setattr(i,k,new_val)
                    except StopIteration: # assume is an choice data type
                        pass
            #print(f'differential post if = {path}.{k} = {getattr(i,k)}')
    summ_file = get_summary(profile_id,sd.differential.element,constraints)
    summ_file = summ_file.replace('\r\n', '\n')
    summ_file = '\n'.join([s for s in summ_file.splitlines() if s]) # remove empty lines
    summ_file = summ_file.replace('####', '\n####')  #add a line before Must Supports

    f_name = f'{profile_id}-summary.md'
    # save in pages folder
    #ig_source_path = ''  # temp folder
    path = Path.cwd() / ig_source_path / 'pages' / '_includes' / f_name
    with path.open(mode='w', encoding='utf8', newline='\n') as f:
        f.write(summ_file)
   

========notifications-messageheader=======
====Path = MessageHeader =====
====Path = MessageHeader.eventCoding =====
no snapshot element for MessageHeader.eventCoding.label = None assume is an choice data type
MessageHeader.eventCoding MessageHeader.eventCoding
no snapshot element for MessageHeader.eventCoding.short = None assume is an choice data type
MessageHeader.eventCoding MessageHeader.eventCoding
no snapshot element for MessageHeader.eventCoding.sliceName = None assume is an choice data type
MessageHeader.eventCoding MessageHeader.eventCoding
====Path = MessageHeader.destination =====
====Path = MessageHeader.sender =====
====Path = MessageHeader.author =====
====Path = MessageHeader.source =====
====Path = MessageHeader.responsible =====
====Path = MessageHeader.response =====
====Path = MessageHeader.focus =====
====Path = MessageHeader.focus.reference =====
============== file_name = notifications-messageheader-summary.md ================




**MessageHeader**
#### Summary of the Mandatory Requirements


1.  A  Coding  in `MessageHeader.eventCoding`
with an [extensible](http://hl7.org/fhir/R4/terminologies.html#extensible)
 binding to [Da Vinci Notification Event ValueSet](ValueSet-notification-event.html)
1.  A  Source  in `MessageHeader.source`

1. One or more Focus References  in `MessageHeader.focus`

      - which must have a  string value  in `MessageHeader.focus.reference`

#### Summary of the Must Support Requirements


1.  A  Destination  in `MessageHeader.destination`

1.  A Sender Reference  in `MessageHeader.sender`

1.  An Author Reference  in `MessageHeader.author`

1.  A Responsible Reference  in `MessageHeader.responsible`

#### Summary of the Unsupported Elements


1. `MessageHeader.response`

========notifications-graphdefinition=======
====Path = GraphDefinition =====
====Path = GraphDefinition.link =====
====Path = GraphDefinition.link.extension =====
============== file_name = notifications-graphdefinition-summary.md ================




**GraphDefinition**


#### Summary of the Must Support Requirements


1. One or more  Links  in `GraphDefinition.link`

   - which should have at least  a [Must-Support](StructureDefinition-extension-mustSupport.html) Extension value  in `GraphDefinition.link.extension`



#### Summary of Constraints
1. Name should be usable as an identifier for the module by machine processing applications such as code generation

========notifications-messageheader-src=======
====Path = MessageHeader =====
====Path = MessageHeader.eventCoding =====
no snapshot element for MessageHeader.eventCoding.label = None assume is an choice data type
MessageHeader.eventCoding MessageHeader.eventCoding
no snapshot element for MessageHeader.eventCoding.short = None assume is an choice data type
MessageHeader.eventCoding MessageHeader.eventCoding
no snapshot element for MessageHeader.eventCoding.sliceName = None assume is an choice data type
MessageHeader.eventCoding MessageHeader.eventCoding
====Path = MessageHeader.destination =====
====Path = MessageHeader.sender =====
====Path = MessageHeader.author =====
====Path = MessageHeader.source =====
====Path = MessageHeader.responsible =====
====Path = MessageHeader.response =====
====Path = MessageHeader.focus =====
====Path = MessageHeader.focus.reference =====
============== file_name = notifications-messageheader-src-summary.md ================


...



**Encounter**
#### Summary of the Mandatory Requirements


1.  A Patient Reference  in `Encounter.subject`

#### Summary of the Must Support Requirements


1. One or more  Participants  in `Encounter.participant`

   - which should have an Individual Reference value  in `Encounter.participant.individual`

1. One or more  Locations  in `Encounter.location`

   - which must have a Location Reference value  in `Encounter.location.location`




========notifications-messagedefinition=======
====Path = MessageDefinition =====
====Path = MessageDefinition.url =====
====Path = MessageDefinition.version =====
====Path = MessageDefinition.name =====
====Path = MessageDefinition.title =====
====Path = MessageDefinition.status =====
====Path = MessageDefinition.date =====
====Path = MessageDefinition.publisher =====
====Path = MessageDefinition.description =====
====Path = MessageDefinition.eventCoding =====
no snapshot element for MessageDefinition.eventCoding.label = None assume is an choice data type
MessageDefinition.eventCoding MessageDefinition.eventCoding
no snapshot element for MessageDefinition.eventCoding.short = None assume is an choice data type
MessageDefinition.eventCoding MessageDefinition.eventCoding
no snapshot element for MessageDefinition.eventCoding.min = None assume is an choice data type
MessageDefinition.eventCoding MessageDefinition.eventCoding
no snapshot element for MessageDefinition.eventCoding.max = None 



**MessageDefinition**
#### Summary of the Mandatory Requirements


1.  An  uri  in `MessageDefinition.url`

1.  A  code  in `MessageDefinition.status`
with a [required](http://hl7.org/fhir/R4/terminologies.html#required)
 binding to [PublicationStatus](http://hl7.org/fhir/ValueSet/publication-status|4.0.1)
1.  A  dateTime  in `MessageDefinition.date`

1.  A  code  in `MessageDefinition.category`  = the fixed value  "notification"
1. One or more  Focuses  in `MessageDefinition.focus`
 with the following constraints: *Max must be postive int or **
      - which must have a  code value  in `MessageDefinition.focus.code`
with a [required](http://hl7.org/fhir/R4/terminologies.html#required)
 binding to [ResourceType](http://hl7.org/fhir/ValueSet/resource-types|4.0.1)
      - which should have a  canonical value  in `MessageDefinition.focus.profile`

1. One or more  canonicals  in `MessageDefinition.graph`

#### Summary of the Must Support Requirements


1.  A  string  in `MessageDefinition.version`

1.  A  string  in `MessageDefinition.name`

1.  A  string  in `MessageDefinition.title`

1.  A  string  in `MessageDefinition.publisher`

1.  A  markdown  in `MessageDefinition.description`

#### Summary of the Unsupported Elements


1. `MessageDefinition.responseRequired`
1. `MessageDefinition.allowedResponse`
#### Summary of Constraints
1. Name should be usable as an identifier for the module by machine processing applications such as code generation
1. Max must be postive int or *

========admit-discharge-notification-condition=======
====Path = Condition =====
====Path = Condition.encounter =====
============== file_name = admit-discharge-notification-condition-summary.md ================




**Condition**
#### Summary of the Mandatory Requirements


1.  An Encounter Reference  in `Condition.encounter`





#### Summary of Constraints
1. Condition.clinicalStatus SHALL be present if verificationStatus is not entered-in-error and category is problem-list-item
1. If condition is abated, then clinicalStatus must be either inactive, resolved, or remission
1. Condition.clinicalStatus SHALL NOT be present if verification Status is entered-in-error
1. A code in Condition.category SHOULD be from US Core Condition Category Codes value set.